In [2]:
## 入出力データ情報
Pass_In    = "Nakano/"
#Pass_Out   = "n/"
Input_File = "入力データ/"
Input_File2 = "空調機別24hデータ/"
Input_Feature = "特徴量データ(2021)/"

Output_File = "出力データ/"
out_clust_data = "クラスタリング結果/"
out_clust_data2 = '時系列クラスタリング(クラスタ数4)/'
clust_number = "クラスタ数4/"
clust_number1 = "真クラスタ数4/"

pattern_1 = "電力消費パターン分析①（定格で正規化のみ）/"
pattern_2 = "電力消費パターン分析②（各空調機の上位値で正規化のみ）/"
Input_data = "UN中野_ポイントリスト.xlsx"
Input_data1 = "UN中野ビル基本データ.csv"
Input_data2 = "UN中野元データ.csv"

year_File = "年間/"
summer_File = "夏期/"
winter_File = "冬期/"

#本分析使用データ：2021/04/01～2022/03/31
Year = '2021'
Term = [Year+'-04-01',str(int(Year)+1)+'-03-31']
Y_sta = ['2018-04-01 00:00:00','2019-04-01 00:00:00','2020-04-01 00:00:00','2021-04-01 00:00:00']
Y_fin = ['2019-03-31 23:00:00','2020-03-31 23:00:00','2021-03-31 23:00:00','2022-03-31 23:00:00']

## 初期装備

In [4]:
## 必要ライブラリ
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import japanize_matplotlib
from matplotlib import gridspec
import matplotlib as mpl
import datetime
import matplotlib.dates as mdates
from datetime import time
from sklearn.cluster import KMeans
from datetime import time
from sklearn import cluster, preprocessing
from sklearn.model_selection import train_test_split
import seaborn as sns
from collections import Counter
import matplotlib.dates as mdates
import os
from ipywidgets import interact
import ipywidgets as widgets

#カレンダー
from dateutil.relativedelta import relativedelta
from matplotlib.patches import Polygon
import calmap

#機械学習
from sklearn.linear_model import LinearRegression # 重回帰
from sklearn.ensemble import RandomForestRegressor #ランダムフォレスト
from sklearn.svm import SVR #サポートベクトルマシン
import lightgbm as lgb #LightGBMライブラリ
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import IsolationForest
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.ensemble import RandomForestClassifier
import jaconv

#深層学習
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Activation
#from keras.layers.recurrent import LSTM
from tensorflow.keras.layers import LSTM
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import PolynomialFeatures
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

#250414 メモリリークの防止
import os
os.environ["OMP_NUM_THREADS"] = "2"

In [5]:
## グラフの定義
## 基本
def graph_fig(fig, ax):
    fig.patch.set_facecolor('white')
    fig.patch.set_alpha(1)
    ax.patch.set_facecolor('lightgrey')
    ax.patch.set_alpha(0.1)
    return fig, ax

def graph_axis(ax):
    # grid line
    ax.yaxis.grid(True, which = 'major', linestyle = '-', color = 'grey', alpha=0.8)
    ax.set_axisbelow(True)
    # axis color
    ax.spines['bottom'].set_color('black')
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.tick_params(axis = 'x', colors ='black',length=5, direction='in')
    ax.tick_params(axis = 'y', colors = 'black',length=0)
    ax.xaxis.label.set_color('black')
    ax.yaxis.label.set_color('black')   
    return ax

def graph_axis2(ax):
    # grid line
    ax.xaxis.grid(True, which = 'major', linestyle = '-', color = 'grey', alpha=0.8)
    ax.set_axisbelow(True)
    # axis color
    ax.spines['bottom'].set_color('black')
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.tick_params(axis = 'x', colors ='black',length=5, direction='in')
    ax.tick_params(axis = 'y', colors = 'black',length=0)
    ax.xaxis.label.set_color('black')
    ax.yaxis.label.set_color('black')   
    return ax

def graph_xy(ax, font, xlabel, ylabel, x_min, x_max, y_min, y_max):
    plt.rcParams["font.size"] = font
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    plt.xlim([x_min, x_max])
    plt.ylim([y_min, y_max])
    return ax

def graph_xy2(ax, font, xlabel, ylabel, x_min, x_max, y_min, y_max):
    plt.rcParams["font.size"] = font
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_xlim([x_min, x_max])
    ax.set_ylim([y_min, y_max])
    return ax

def graph_x(ax, font, xlabel, ylabel, x_min, x_max):
    plt.rcParams["font.size"] = font
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_xlim([x_min, x_max])
    #plt.ylim([y_min, y_max])
    return ax

def graph_none(fig, ax):
    ax.spines['bottom'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.axis("off")
    plt.ylim([10, 20])
    return fig, ax

def graph_grid(ax0): 
    # grid line
    ax0.yaxis.grid(True, which = 'major', linestyle = '-', color = 'grey', alpha=0.8)
    ax0.set_axisbelow(True)
    # axis color
    ax0.set_axisbelow(True)
    ax0.spines['bottom'].set_color('black')
    ax0.spines['top'].set_visible(False)
    ax0.spines['left'].set_visible(False)
    ax0.spines['right'].set_visible(False)
    ax0.tick_params(axis = 'x', colors ='black',length=5, direction='in')
    ax0.tick_params(axis = 'y', colors = 'black',length=0)
    ax0.xaxis.label.set_color('black')
    ax0.yaxis.label.set_color('black')
    return ax0
def graph_xy1(ax, font, xlabel, ylabel, x_min, x_max, y_min, y_max):
    plt.rcParams["font.size"] = font
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_xlim([x_min, x_max])
    ax.set_ylim([y_min, y_max])
    return ax

In [6]:
## グラフの定義（一部使用）
## 建物分け
def graph_split_label(ax, label_1st_num, axis_1st_num, list_n, split_label, font_size, annotate_y, text_y, y_min, y_max):
    data_num  = axis_1st_num #初期xの値
    ax.plot([data_num-0.5, data_num-0.5], [y_min, y_max], c='black', alpha=0.75, linestyle='dotted',linewidth=1.5)
    for s in range(len(split_label)):
        new_num = len(list_n[list_n == label_1st_num + s])
        start = [data_num-0.5, annotate_y]
        data_num = data_num + new_num
        end =   [data_num-0.5, annotate_y]
        ax.plot([data_num-0.5, data_num-0.5], [y_min, y_max], c='black', alpha=0.75, linestyle='dotted',linewidth=1.5)
        ax.text((start[0]+end[0])*0.5, text_y, split_label[s], ha='center', weight='bold', size=font_size)
        ax.annotate('', xy=start, xytext=end, arrowprops=dict(arrowstyle='<|-|>, head_width=0.3',lw=2, 
                    facecolor='white', edgecolor='black', alpha=0.75))
    return ax

## 月ラベル
def Month_Label(M_Input, Train, Test):

    Month = ['12月','1月','2月','3月','4月','5月','6月','7月','8月','9月','10月','11月']
    Month_New = []
    m_sta = M_Input[M_Input.index[0]]
    for s in range(int(len(M_Input)/30)):
        Month_New.append(Month[(m_sta+s) %len(Month)])
    return Month_New

## plot（分かりにくい）
def Plot_Scater(ax,s,graph_label,Input_Data,DataSet,Tra_or_Tes, mark_col, edge_col,mark,m_size,m_alpha,l_width,legend,label):
    Line = ax.scatter(Input_Data.iloc[Tra_or_Tes.index,:]['日時'], DataSet.iloc[Tra_or_Tes.index,s], c=mark_col, edgecolors=edge_col, marker=mark,
          s=m_size , alpha=m_alpha, linestyle='-', linewidths=l_width)
    legend.append(Line)
    label.append(graph_label[s])
    return ax, legend, label

## plot（分かりにくい）
def Plot_Scater2(ax,s,graph_label,Input_Data,DataSet,Tra_or_Tes, mark_col, edge_col,mark,m_size,m_alpha,l_width,legend,label):
    Line = ax.scatter(Input_Data.iloc[Tra_or_Tes.index,:]['日時'], DataSet.iloc[Tra_or_Tes.index,:]['実績']-DataSet.iloc[Tra_or_Tes.index,s], c=mark_col, edgecolors=edge_col, marker=mark,
          s=m_size , alpha=m_alpha, linestyle='-', linewidths=l_width)
    legend.append(Line)
    label.append(graph_label[s])
    return ax, legend, label

## plot（分かりにくい）
def Plot_Scater3(ax,s,graph_label,Input_Data,DataSet,mark_col,edge_col,mark,m_size,m_alpha,l_width,legend,label):
    Line = ax.scatter(Input_Data['日時'], DataSet.iloc[:,s], c=mark_col, edgecolors=edge_col, marker=mark,
          s=m_size , alpha=m_alpha, linestyle='-', linewidths=l_width)
    legend.append(Line)
    label.append(graph_label[s])
    return ax, legend, label

In [7]:
## カラーの名称
light_c = ["#FF4500", "#1E90FF", "#32CD32", "#DA70D6", "#FFD700", "#708090", "#DEB887"]
dark_c  = ["#B22222", "#0000CD", "#006400", "#9932CC", "#B8860B", "#696969", "#A0522D"]

           #ピンク系  #オレンジ系 #イエロー系 #紫系     #緑系       #赤系     #青系
light_b = ["#FF69B4", "#FFA500", "#FFFACD", "#DDA0DD", "#98FB98", "#F08080", "#87CEFA"]
dark_b =  ["#C71585", "#FF4500", "#F0E68C", "#BA55D3", "#32CD32", "#DC143C", "#1E90FF"]

## スタイルの名称
markers = ['o', '^', 's', 'v', '^', 'D']
l_styles = ['-', ':', '-.', '--']

In [8]:
## 正規化関数
def zscore(x, axis = None):
    xmean = x.mean(axis=axis, keepdims=True)
    xstd  = np.std(x, axis=axis, keepdims=True)
    zscore = (x-xmean)/xstd
    return zscore

## 分析データ読み込み・作成

In [10]:
#2018-04-01～2022-03-31元データ
data = pd.read_csv(Pass_In + Input_File+Input_data2,encoding='cp932',index_col=0,parse_dates=True)[2:]
#データ形式変更
for i in range(len(data.columns)):
    data.iloc[:,i] = data.iloc[:,i].astype('float64')

C:\Users\Busy_Lake\AppData\Local\Temp\ipykernel_14560\2424032524.py:2: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,25

In [11]:
#ポイントリスト読み込み
Point_list = pd.read_csv('Nakano/入力データ/Nakanoポイントリスト.csv',encoding='cp932')
Point_list = Point_list.astype(str)

In [12]:
##各種ラベル名
EHP_name = ['EHPK-1','EHPK-2','EHPK-3','GHEX-1','GHEX-2','GHEX-3','GHEX-4','GHEX-5','EHP1-1','EHP1-2','EHP1-3',
 'EHP2-1','EHP2-2','EHP2-3','EHP2-4','EHP2-5','EHP2-6','EHP3-1','EHP3-2','EHP3-3','EHP3-4','EHP3-5','EHP3-6',
 'EHP4-1','EHP4-2','EHP4-3','EHP4-4','EHP4-5','EHP4-6','EHP5-1','EHP5-2','EHP5-3',
 'EHP6-1','EHP6-2','EHP6-3']

In [13]:
##定格電力消費・能力データ読み込み
EHP_list = pd.read_csv('Nakano/入力データ/空調機定格.csv',index_col=0)
EHP_list = EHP_list[['冷房定格電力kW','暖房定格電力kW']]

In [14]:
##各種データ集計
#フロア別集計
Data_F =pd.DataFrame()
for i in range(1,7):#1から6階
    a = data[Point_list[Point_list['フロア別']==str(i)]['ポイント番号']].T.sum()
    Data_F[str(i)+'階'] = a
Data_F['合計'] = data['312000957']
    
#用途別集計
Data_App = pd.DataFrame()
Data_App['空調'] = data[Point_list[Point_list['用途別']=='空調']['ポイント番号']].T.sum()
Data_App['照明'] = data[Point_list[Point_list['用途別']=='照明']['ポイント番号']].T.sum()
Data_App['OA機器'] = data[Point_list[Point_list['用途別']=='OA機器']['ポイント番号']].T.sum()
Data_App['自動販売機'] = data['312001111']
Data_App['その他'] = data['312000957']-(Data_App['空調'] + Data_App['照明'] + Data_App['OA機器']+data['312001111'])
Data_App['合計'] = data['312000957']


#部門別集計
Data_Depa = pd.DataFrame()
Data_Depa['共用'] = data[Point_list[Point_list['部門別']=='共用']['ポイント番号']].T.sum()+data['312001111']
Data_Depa['専有'] = data['312000957']-Data_Depa['共用']
Data_Depa['合計'] = data['312000957']

#クラスタリング対象データ(35こ)
Data_clust = data[Point_list[Point_list['クラスタリング対象']=='1']['ポイント番号']]

#BL対象データ(19こ)
Data_bl = data[Point_list[Point_list['BL対象']=='1']['ポイント番号']]

In [15]:
###各年度集計

#フロア別各年度
F_2018 = Data_F[Y_sta[0]:Y_fin[0]]
F_2019 = Data_F[Y_sta[1]:Y_fin[1]]
F_2020 = Data_F[Y_sta[2]:Y_fin[2]]
F_2021 = Data_F[Y_sta[3]:Y_fin[3]]

##用途別集計各年度
A_2018 = Data_App[Y_sta[0]:Y_fin[0]]
A_2019 = Data_App[Y_sta[1]:Y_fin[1]]
A_2020 = Data_App[Y_sta[2]:Y_fin[2]]
A_2021 = Data_App[Y_sta[3]:Y_fin[3]]

#部門別集計各年度
D_2018 = Data_Depa[Y_sta[0]:Y_fin[0]]
D_2019 = Data_Depa[Y_sta[1]:Y_fin[1]]
D_2020 = Data_Depa[Y_sta[2]:Y_fin[2]]
D_2021 = Data_Depa[Y_sta[3]:Y_fin[3]]

##クラスタリング対象データ各年度
clust_2018 = Data_clust[Y_sta[0]:Y_fin[0]]
clust_2019 = Data_clust[Y_sta[1]:Y_fin[1]]
clust_2020 = Data_clust[Y_sta[2]:Y_fin[2]]
clust_2021 = Data_clust[Y_sta[3]:Y_fin[3]]

##BL対象データ各年度
BL_2018 = Data_bl[Y_sta[0]:Y_fin[0]]
BL_2019 = Data_bl[Y_sta[1]:Y_fin[1]]
BL_2020 = Data_bl[Y_sta[2]:Y_fin[2]]
BL_2021 = Data_bl[Y_sta[3]:Y_fin[3]]

### 季節別データ定義

In [17]:
#春夏秋冬の定義したい(夏期・中間期・冬期の定格を定義するため)

#2018-04-01～2022-03-31元データ
data_all = pd.read_csv(Pass_In + Input_File+Input_data2,encoding='cp932',index_col=0,parse_dates=True)[2:].copy()

#データ形式変更
for col in data_all.columns:
    data_all.loc[:, col] = data_all[col].astype('float64')

Year = '2021'
season_sta = [Year+'-04-01 00:00', Year+'-06-01 00:00', Year+'-10-01 00:00', Year+'-12-01 00:00']
season_fin = [Year+'-05-31 23:00', Year+'-09-30 23:00', Year+'-11-30 23:00', str(int(Year)+1)+'-03-31 23:00']

##2021年外気温データ
Tem_2021 =data_all[Y_sta[3]:Y_fin[3]]['312000756'].resample('D').mean()


#空調・外気温度（季節別）
Spring = clust_2021[season_sta[0]:season_fin[0]]
Spring['外気温度'] = Tem_2021[season_sta[0]:season_fin[0]]
Summer = clust_2021[season_sta[1]:season_fin[1]]#夏
Summer['外気温度'] = Tem_2021[season_sta[1]:season_fin[1]]
Autumn = clust_2021[season_sta[2]:season_fin[2]]
Autumn['外気温度'] = Tem_2021[season_sta[2]:season_fin[2]]
Winter = clust_2021[season_sta[3]:season_fin[3]]#冬
Winter['外気温度'] = Tem_2021[season_sta[3]:season_fin[3]]
Mid = pd.concat([Spring,Autumn],axis=0)#中間期

#使用データ：Spring、Summer、Mid

C:\Users\Busy_Lake\AppData\Local\Temp\ipykernel_14560\584819382.py:4: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254

In [18]:
#季節別クラスタリング使用データ：、Summer、Mid、Winter(時刻別)
Su = Summer.drop('外気温度',axis=1)#夏
Su.columns = EHP_name
Mi = Mid.drop('外気温度',axis=1)#中間期
Mi.columns = EHP_name
Wi = Winter.drop('外気温度',axis=1)#冬
Wi.columns = EHP_name

In [19]:
Mi['EHP3-4']

2021-04-01 00:00:00    0.0
2021-04-01 01:00:00    0.0
2021-04-01 02:00:00    0.0
2021-04-01 03:00:00    1.0
2021-04-01 04:00:00    0.0
                      ... 
2021-11-30 19:00:00    0.0
2021-11-30 20:00:00    0.0
2021-11-30 21:00:00    1.0
2021-11-30 22:00:00    0.0
2021-11-30 23:00:00    0.0
Name: EHP3-4, Length: 2928, dtype: object

### 定格電力で正規化データ作成

In [21]:
##空調機データ正規化（夏期=Su・冬期=Wi）
##夏期
Su_norm = pd.DataFrame()
for i in range(len(EHP_name)):
    Su_norm[EHP_name[i]] = Su.iloc[:,i]/EHP_list['冷房定格電力kW'][i]
Su_norm = Su_norm.round(2)
##冬期
Wi_norm = pd.DataFrame()
for i in range(len(EHP_name)):
    Wi_norm[EHP_name[i]] = Wi.iloc[:,i]/EHP_list['暖房定格電力kW'][i]
Wi_norm = Wi_norm.round(2)

C:\Users\Busy_Lake\AppData\Local\Temp\ipykernel_14560\4251615902.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Su_norm[EHP_name[i]] = Su.iloc[:,i]/EHP_list['冷房定格電力kW'][i]
C:\Users\Busy_Lake\AppData\Local\Temp\ipykernel_14560\4251615902.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Su_norm[EHP_name[i]] = Su.iloc[:,i]/EHP_list['冷房定格電力kW'][i]
C:\Users\Busy_Lake\AppData\Local\Temp\ipykernel_14560\4251615902.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value 

In [22]:
##中間期
#中間期定格電力定義（仮）
mid_fix = (EHP_list['暖房定格電力kW']+EHP_list['冷房定格電力kW'])/2
#春
Sp = Spring.drop('外気温度',axis=1)
Sp_norm = pd.DataFrame()
for i in range(len(EHP_name)):
    Sp_norm[EHP_name[i]] = Sp.iloc[:,i]/mid_fix[i]
Sp_norm = Sp_norm.round(2)

#秋
Au = Autumn.drop('外気温度',axis=1)
Au_norm = pd.DataFrame()
for i in range(len(EHP_name)):
    Au_norm[EHP_name[i]] = Au.iloc[:,i]/mid_fix[i]
Au_norm = Au_norm.round(2)

C:\Users\Busy_Lake\AppData\Local\Temp\ipykernel_14560\2455969603.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Sp_norm[EHP_name[i]] = Sp.iloc[:,i]/mid_fix[i]
C:\Users\Busy_Lake\AppData\Local\Temp\ipykernel_14560\2455969603.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Sp_norm[EHP_name[i]] = Sp.iloc[:,i]/mid_fix[i]
C:\Users\Busy_Lake\AppData\Local\Temp\ipykernel_14560\2455969603.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc

In [23]:
##年間正規化データ作成（仮）
Year_norm = pd.concat([Sp_norm,Su_norm,Au_norm,Wi_norm ],axis=0)
#Year_norm.to_csv(Pass_In+Input_File+'入力比データ.csv',encoding='cp932')

In [24]:
##各空調機の空調日・非空調日判定

## 各空調機の稼働状況(0 or 1):時刻別
EHP_count = pd.DataFrame(index=Year_norm.index)
data = Year_norm.fillna(method='bfill')#正規化済み

for s in range(len(data.columns)):
    A = []
    for i in range(len(data)):
        if data[data.columns[s]][i]>=0.05:#定格の5%以上発揮→[空調日]
            A.append(1)
        elif data[data.columns[s]][i]<0.05:#[非空調日]
            A.append(0)
    EHP_count[EHP_name[s]] =A
    

EHP_count['日時'] = pd.to_datetime(EHP_count.index)
#DataSet['日付'] = pd.to_datetime(DataSet.index)
EHP_count['日付'] = EHP_count['日時'].dt.date

C:\Users\Busy_Lake\AppData\Local\Temp\ipykernel_14560\1200912800.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = Year_norm.fillna(method='bfill')#正規化済み
C:\Users\Busy_Lake\AppData\Local\Temp\ipykernel_14560\1200912800.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = Year_norm.fillna(method='bfill')#正規化済み
C:\Users\Busy_Lake\AppData\Local\Temp\ipykernel_14560\1200912800.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if data[data.columns[s]][i]>=0.05:#定格の5%以上発揮→[空調日]
C:\Us

# 各種分析

## 入力比パターン分析（定格で正規化のみ）

### 時系列クラスタリングによる分類（入力比）

In [28]:
##データ整理
Frame = pd.read_csv(Pass_In+Input_File+'入力比データ.csv',encoding='cp932',index_col=0,parse_dates=True).fillna(0).resample('D').sum()
DataSet = Frame.T


In [29]:
##クラスタリング
learning_label = 0
n_clusters =2

if learning_label == 0:
    km = KMeans(n_clusters=n_clusters,init='k-means++', n_init=10, max_iter=300, tol=1e-04, random_state=0)
    y_km = km.fit_predict(Frame)

Result_km = pd.DataFrame()
Result_km['設備名'] = EHP_name
Result_km['クラスタ番号'] = y_km

C:\Users\Busy_Lake\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


ValueError: Length of values (365) does not match length of index (35)

In [ ]:
##グラフ化
fig = plt.figure(figsize = (15,3.5),dpi=200)

plt.rcParams["font.size"] = 13  ##文字のおおきさ
fig.patch.set_facecolor('white')  ##全体の背景の色
fig.patch.set_alpha(1)  ##透明度、alpha=1無透明

ax0 = fig.add_subplot(1,1,1)
ax0.patch.set_facecolor('lightgrey')   ##patchは背景、背景の色決め
ax0.patch.set_alpha(0.1)  ##背景の透明度

label=['Cluster0','Cluster1','Cluster2','Cluster3']

##クラスタ4
for s in range(n_clusters):
    Data = DataSet[y_km==s].T
    Data_mean = Data.T.mean()
    ##平均と全ての電力消費を同時プロット
    ax0.plot(Data.index, Data, color=plt.rcParams['axes.prop_cycle'].by_key()['color'][s], alpha=0.2)##個々の電力消費は薄め
    ax0.plot(Data.index, Data_mean, color=plt.rcParams['axes.prop_cycle'].by_key()['color'][s],
             alpha=1, linewidth=2,label=label[s])##平均は濃く


#plt.ylim([0,1])

###axに関して美しく作成
# grid line
ax0.yaxis.grid(True, which = 'major', linestyle = '-', color = 'gray', alpha=0.8)##y軸グリッドに関して
# axis color
ax0.set_axisbelow(True)
ax0.spines['bottom'].set_color('black')##下枠色指定
ax0.spines['top'].set_visible(False)##上枠を消す
ax0.spines['left'].set_visible(False)##左枠を消す
ax0.spines['right'].set_visible(False)##右枠を消す
ax0.tick_params(axis = 'x', colors ='black',length=5, direction='in')##x軸のパラメータに関して
ax0.tick_params(axis = 'y', colors = 'black',length=0)##y軸のパラメータに関して
ax0.xaxis.label.set_color('black')
ax0.yaxis.label.set_color('black')
ax0.legend(bbox_to_anchor = (0.5,1.3), loc = 'lower center', ncol = 4, fontsize = 15,facecolor='white')##labelを設定する

#plt.title('空調電力消費量\nクラスタリング結果\n(2021年)',fontsize = 14) # タイトル
plt.xlabel('') # x軸ラベル
plt.ylabel('入力比') # y軸ラベル

In [ ]:
##データ準備
Holi = DataSet_pat[0].mean()
Sum = DataSet_pat[1].mean()
Win = DataSet_pat[2].mean()
Sum_p = DataSet_pat[3].mean()
Win_p = DataSet_pat[4].mean()

mean = (Holi+Sum+Win+Sum_p+Win_p)/5
col = pat_label
FRAME = pd.concat([Holi,Sum,Win,Sum_p,Win_p],axis=1)
FRAME.columns=col
FRAME['入力比平均'] = mean
COL = ['入力比平均','休日', '夏期空調日', '冬期空調日', '夏期ピーク発生日', '冬期ピーク発生日']
FRAME = FRAME[COL]

In [ ]:
##類型化結果反映
##データ読み込み
Result_km = Result_km.set_index('設備名')
Per = FRAME
X=['入力比平均','休日', '夏期空調日', '冬期空調日', '夏期ピーク\n発生日', '冬期ピーク\n発生日']
# 正規化データ(DataをData_Scaleに変えただけ)(全体のピーク値を1.0)
Per_Scale = pd.DataFrame()
Per_Scale = (Per - Per.values.min()) / (Per.values.max() - Per.values.min())
##データ並び替え
Per_Scale = Per_Scale.sort_values('入力比平均', ascending=True)
Per_Scale = Per_Scale.sort_values('冬期ピーク発生日', ascending=True)
Per_Scale = Per_Scale.sort_values('夏期ピーク発生日', ascending=True)
Per_Scale = Per_Scale.sort_values('夏期空調日', ascending=True)
Per_Scale = Per_Scale.sort_values('冬期空調日', ascending=True)

#Result_km = Result_km.set_index('建物名',drop=True)
Result_km = Result_km.T[Per_Scale.index].T

#バブルプロット
#4つの変数（x軸、y軸、バブルの大きさ、バブルの色）を描画できる！
col = ['Tomato', 'MediumPurple', 'lightGreen','DodgerBlue']
col2 = ['OrangeRed', 'Indigo', 'Green','darkblue']
lab =  ['No.0', 'No.1', 'No.2','No.3']

plt.rcParams["font.size"]=16
fig = plt.figure(figsize=(11,11),dpi=200)
mark_s = [0.0005]

for energy in range(len(Per_Scale.columns[::-1])):
    for build in range(len(Per_Scale.index)):
        plt.scatter(X[energy], Per_Scale.index[build],s=Per_Scale[Per_Scale.columns[energy]][Per_Scale.index[build]]/mark_s[energy %len(mark_s)], 
                    alpha=0.5, linewidths=2,color=col[Result_km['クラスタ番号'][build]],
                    edgecolors=col2[Result_km['クラスタ番号'][build]])
        
plt.grid(color='gray', linestyle='dotted', linewidth=1)     
#plt.title("エネルギー種別\n原単位[MJ/m2・年]")
plt.xlabel("")
plt.ylabel("空調室外機名") 

##凡例は別で作成
fig, ax0 = plt.subplots(ncols=1, figsize=(10,2),dpi=200)
size = [200,200,200,200]
for s in range(4):
    plt.scatter(0,0,s=size[s],facecolor=col[s],edgecolor=col2[s],label=lab[s], alpha=0.7, linewidths=2)
plt.legend(ncol=4,title='', bbox_to_anchor=(1.35, 1), loc='upper left',labelspacing=1,borderpad=1)
graph_none(fig, ax0)

### パターン分析（定格で正規化のみ）入力比

In [ ]:
## データID
n_clusters     =  4
outliers_id    =  1
learning_label =  0

#各ラベル定義
Year = '2021'
season_sta = [Year+'-04-01 00:00', Year+'-06-01 00:00', Year+'-10-01 00:00', Year+'-12-01 00:00']
season_fin = [Year+'-05-31 23:00', Year+'-09-30 23:00', Year+'-11-30 23:00', str(int(Year)+1)+'-03-31 23:00']

#使用データ：Year_norm

In [ ]:
## データセット指定
DataSet = pd.read_csv(Pass_In+Input_File+'入力比データ.csv',encoding='cp932',index_col=0,parse_dates=True).fillna(0)
##各種ラベル名
EHP_name = ['EHPK-1','EHPK-2','EHPK-3','GHEX-1','GHEX-2','GHEX-3','GHEX-4','GHEX-5','EHP1-1','EHP1-2','EHP1-3',
 'EHP2-1','EHP2-2','EHP2-3','EHP2-4','EHP2-5','EHP2-6','EHP3-1','EHP3-2','EHP3-3','EHP3-4','EHP3-5','EHP3-6',
 'EHP4-1','EHP4-2','EHP4-3','EHP4-4','EHP4-5','EHP4-6','EHP5-1','EHP5-2','EHP5-3',
 'EHP6-1','EHP6-2','EHP6-3']
## 空調機ラベル
G_LABEL = EHP_name

## データインデックス
_from = season_sta[0]
_to   = season_fin[3]

date_index = pd.date_range(start=_from, end=_to, freq='H')

## 分析データ処理
DataSet['日時'] = pd.to_datetime(DataSet.index)
#DataSet['日付'] = pd.to_datetime(DataSet.index)
DataSet['日付'] = DataSet['日時'].dt.date
DataSet = DataSet[(_from<=DataSet['日時'])&(DataSet['日時']<=_to)]

In [ ]:
##祝日作り
import jpholiday
analys_day = []
for i in range(int(len(date_index)/24)):
    analys_day.append(date_index[0].date() + datetime.timedelta(days=i)) #analys_day = "yyyy, mm, dd" (date_indexの最初の日付から数える)

## DATE = "yyyy, mm, dd"  日付は８桁文字列の形式
def isBizDay(DATE):
        #Date = datetime.date(int(DATE[0:4]), int(DATE[4:6]), int(DATE[6:8]))
    Date = DATE
    if Date.weekday() >= 5 or jpholiday.is_holiday(Date):
        return 0
    else:
        return 1    

## 日単位
holiday_list_day = [] #日にちの0or1データ
for i in range(len(analys_day)):
    s = isBizDay(analys_day[i])
    holiday_list_day.append(s)

## 時間単位
holiday_list_hour = [] #時間の0or1データ
for s in range(len(holiday_list_day)):
    for i in range(24):
        holiday_list_hour.append(holiday_list_day[s])

DataSet['休日'] = holiday_list_hour

In [ ]:
## 各パターン作成
pat_label = ['休日','夏期','冬期','夏期空調日','冬期空調日','非空調日','ピーク発生日']
pat_label = ['休日','夏期空調日','冬期空調日','非空調日','ピーク発生日']
pat_label = ['休日','夏期空調日','冬期空調日','ピーク発生日']
pat_label = ['休日','夏期','冬期','ピーク発生日']
pat_label = ['休日','夏期空調日','冬期空調日','夏期ピーク発生日','冬期ピーク発生日']
#pat_label = ['休日','夏期空調日','冬期空調日','冬期ピーク発生日']

season_s = [Year+'-04-01 00:00', Year+'-06-01 00:00', Year+'-10-01 00:00', Year+'-01-01 00:00']
season_f = [Year+'-05-31 23:00', Year+'-08-30 23:00', Year+'-11-30 23:00', str(int(Year)+1)+'-02-28 23:00']

DataSet_pat = []

## 休日
Frame = pd.DataFrame()
for i in range(24):
    DataSet_holi = DataSet[DataSet['休日']==0]
    Frame[i] = DataSet_holi.set_index('日時').loc[time(i)].mean()[G_LABEL].round(2)
DataSet_pat.append(Frame.T)

##夏期平日空調日
Frame = pd.DataFrame()
data = DataSet[DataSet['休日']==1][(season_s[1]<=DataSet['日時'])&(DataSet['日時']<=season_f[1])].fillna(0)

for s in range(len(G_LABEL)):
    index = data[G_LABEL[s]][data[G_LABEL[s]] >=0.05].index
    index = pd.DatetimeIndex(index.date)
    index = list(set(index))
    R = pd.DataFrame()
    for i in range(len(index)):
        a = data[data['日付'] == index[i]][G_LABEL[s]]
        R = pd.concat([R,a])
        A = []
        for i in range(24):
            r = R.loc[time(i)].mean()[0]
            A.append(r)
    Frame[G_LABEL[s]] = A
Frame = Frame.T.round(2).fillna(0)
DataSet_pat.append(Frame.T) 

##冬期平日空調日
Frame = pd.DataFrame()
data = DataSet[DataSet['休日']==1][(season_s[3]<=DataSet['日時'])&(DataSet['日時']<=season_f[3])].fillna(0)

for s in range(len(G_LABEL)):
    index = data[G_LABEL[s]][data[G_LABEL[s]] >=0.05].index
    index = pd.DatetimeIndex(index.date)
    index = list(set(index))
    R = pd.DataFrame()
    for i in range(len(index)):
        a = data[data['日付'] == index[i]][G_LABEL[s]]
        R = pd.concat([R,a])
        A = []
        for i in range(24):
            r = R.loc[time(i)].mean()[0]
            A.append(r)
    Frame[G_LABEL[s]] = A
Frame = Frame.T.round(2).fillna(0)
DataSet_pat.append(Frame.T) 

"""
## 夏期(平日)
Frame = pd.DataFrame()
for i in range(24):
    DataSet_work = DataSet[DataSet['休日']==1][(season_s[1]<=DataSet['日時'])&(DataSet['日時']<=season_f[1])]
    Frame[i] = DataSet_work.set_index('日時').loc[time(i)].mean()[G_LABEL].round(2)
DataSet_pat.append(Frame.T)

## 冬期（平日）
Frame = pd.DataFrame()
for i in range(24):
    DataSet_work = DataSet[DataSet['休日']==1][(season_s[3]<=DataSet['日時'])&(DataSet['日時']<=season_f[3])]
    Frame[i] = DataSet_work.set_index('日時').loc[time(i)].mean()[G_LABEL].round(2)
DataSet_pat.append(Frame.T)


#空調日(8月)
Month_sta = [Year+'-07-01 00:00', Year+'-08-01 00:00', Year+'-9-01 00:00', str(int(Year)+1)+'-01-01 23:00',str(int(Year)+1)+'-02-01 23:00']
Month_fin = [Year+'-07-31 23:00', Year+'-08-31 23:00', Year+'-9-30 23:00', str(int(Year)+1)+'-01-31 23:00',str(int(Year)+1)+'-02-28 23:00']

Frame = pd.DataFrame()
#data = DataSet[(Month_sta[1]<=DataSet['日時'])&(DataSet['日時']<=Month_fin[1])].fillna(0)#正規化済み
data = DataSet[(season_s[1]<=DataSet['日時'])&(DataSet['日時']<=season_f[1])].fillna(0)

for s in range(len(G_LABEL)):
    A = []
    index = data[G_LABEL[s]][data[G_LABEL[s]] >=0.05].index#各設備入力比（正規化電力）平均以上発揮した日を「空調日」として抽出
    a = DataSet.T[index].T[G_LABEL[s]]
    for i in range(24):
        r = a.loc[time(i)].mean()
        A.append(r)
    Frame[G_LABEL[s]] = A
Frame = Frame.T.round(2).fillna(0)
DataSet_pat.append(Frame.T) 

#空調日(2月)
Frame = pd.DataFrame()
#data = DataSet[(Month_sta[3]<=DataSet['日時'])&(DataSet['日時']<=Month_fin[3])].fillna(0)#正規化済み
data = DataSet[(season_s[3]<=DataSet['日時'])&(DataSet['日時']<=season_f[3])].fillna(0)
for s in range(len(G_LABEL)):
    A = []
    index = data[G_LABEL[s]][data[G_LABEL[s]] >=0.05].index#各設備入力比（正規化電力）平均以上発揮した日を「空調日」として抽出
    a = DataSet.T[index].T[G_LABEL[s]]
    for i in range(24):
        r = a.loc[time(i)].mean()
        A.append(r)
    Frame[G_LABEL[s]] = A
Frame = Frame.T.round(2).fillna(0)
DataSet_pat.append(Frame.T) 

#非空調日2
Frame = pd.DataFrame()
data = DataSet.fillna(method='bfill')#正規化済み
#data = EHP_count.resample('D').sum()[G_LABEL]#1が空調日0が非空調日
#B=pd.DataFrame(index=list(range(24)))
for s in range(len(G_LABEL)):
    A = []
    index = data[G_LABEL[s]][data[G_LABEL[s]] <0.05].index#各設備入力比（正規化電力）平均以上発揮した日を「空調日」として抽出
    a = DataSet.T[index].T[G_LABEL[s]]
    for i in range(24):
        r = a.loc[time(i)].mean()#.round(3)
        A.append(r)
    Frame[G_LABEL[s]] = A
Frame = Frame.T.round(2).fillna(0)
DataSet_pat.append(Frame.T)

## ピーク発生日
Frame = pd.DataFrame()
for s in range(len(G_LABEL)):
    data = DataSet.set_index('日時').resample('D').max()[G_LABEL[s]]
    index = data[data == data.max()].index[0]
    Frame[G_LABEL[s]] = DataSet[DataSet['日付'] == index][G_LABEL[s]].reset_index(drop=True)[0:24]
DataSet_pat.append(Frame)
"""

##夏期ピーク発生日
Frame = pd.DataFrame()
for s in range(len(G_LABEL)):
    data = DataSet[(season_s[1]<=DataSet['日時'])&(DataSet['日時']<=season_f[1])].resample('D').max()[G_LABEL[s]]
    index = data[data == data.max()].index[0]
    Frame[G_LABEL[s]] = DataSet[DataSet['日付'] == index][G_LABEL[s]].reset_index(drop=True)[0:24]
DataSet_pat.append(Frame)

##冬期ピーク発生日
Frame = pd.DataFrame()
for s in range(len(G_LABEL)):
    data = DataSet[(season_s[3]<=DataSet['日時'])&(DataSet['日時']<=season_f[3])].resample('D').max()[G_LABEL[s]]
    index = data[data == data.max()].index[0]
    Frame[G_LABEL[s]] = DataSet[DataSet['日付'] == index][G_LABEL[s]].reset_index(drop=True)[0:24]
DataSet_pat.append(Frame)


'''
## 正規化
for i in range(len(G_LABEL)):
    ## 上位0.1%の平均で除算
    a05 = DataSet[G_LABEL[i]].sort_values(ascending=False).head(int(len(DataSet[G_LABEL])*0.001)).mean()
    for s in range(len(DataSet_pat)):
        DataSet_pat[s][G_LABEL[i]] = (DataSet_pat[s][G_LABEL[i]]/a05).round(3)
'''
#ここでの正規化は必要？？

In [ ]:
## 分析結果
## クラスタリング用データセット
DataSet_clust = pd.DataFrame()
for i in range(len(DataSet_pat)):
    DataSet_clust = pd.concat([DataSet_clust, DataSet_pat[i]])
DataSet_clust = DataSet_clust.reset_index(drop=True)
Frame = DataSet_clust.T.fillna(0)

In [ ]:
## クラスタリング
if learning_label == 0:
    km = KMeans(n_clusters=n_clusters,init='k-means++', n_init=10, max_iter=300, tol=1e-04, random_state=0)
elif learning_label == 1:
    km =cluster.SpectralClustering(n_clusters=n_clusters, affinity="nearest_neighbors", n_init=10, random_state=0)
elif learning_label == 2:
    km = cluster.MeanShift()
y_km = km.fit_predict(Frame)
    
Result_km = pd.DataFrame()
Result_km['設備名'] = EHP_name
Result_km['クラスタ番号'] = y_km    
"""
## クラスタ番号の加工    
Cluss, Cluster = [], [1,0,2,3,4]
for i in range(len(Result_km)):
    for s in range(n_clusters):
        if Result_km['クラスタ番号'][i] == s :
            Cluss.append(Cluster[s])
Result_km['クラスタ番号'] = Cluss
"""
Result_km.to_csv(Pass_In+Output_File+out_clust_data+clust_number1+pattern_1+'result.csv',encoding='cp932')
## 表に図示
for s in range(n_clusters):
    name = Result_km[Result_km['クラスタ番号'] == s]
    display(name)
#グラフ用
E = Result_km.sort_values('クラスタ番号').set_index('設備名',drop=True)
#E.to_csv('入力比クラスタリング結果.csv',encoding='cp932')

In [ ]:
## グラフ
fig = plt.figure(figsize=(8,10),dpi=200, tight_layout=True)
for i in range(n_clusters):
    ax=fig.add_subplot(321+i)
    graph_fig(fig, ax)
    for s in range(len(DataSet_pat)):
        build_clust = Result_km[Result_km['クラスタ番号']==i]['設備名']
        Data = DataSet_pat[s][build_clust].T.mean()
        ax.plot(Data.index, Data, color=dark_c[s %len(dark_c)], label=pat_label[s], linewidth=3,linestyle="dotted", 
                 marker=markers[0], markersize=6, markeredgewidth=1, markeredgecolor=dark_c[s %len(dark_c)], markerfacecolor=light_c[s %len(light_c)], alpha=0.7)
    plt.text(1, 1.3, 'No.'+str(i), fontsize=16)
    plt.hlines(1, 0, 24,colors='red', linestyle='dashed', linewidth=1)
    graph_xy(ax, 15, '時間', '入力比', 0,24, 0,1.6)
    graph_axis(ax)


## 凡例のみ表示
fig, ax0 = plt.subplots(ncols=1, figsize=(10,0.1),dpi=200)
for s in range(len(DataSet_pat)):
    ax0.plot(Data.index, Data, color=dark_c[s %len(dark_c)], label=pat_label[s], linewidth=3,linestyle="dotted", 
             marker=markers[0], markersize=7.5, markeredgewidth=1, markeredgecolor=dark_c[s %len(dark_c)], markerfacecolor=light_c[s %len(light_c)], alpha=0.7)
graph_none(fig, ax0)
axlegend1 = ax0.legend(bbox_to_anchor=(0.5, 0.5), loc='center', ncol=1, frameon = True, fontsize = 15, numpoints = 1,facecolor='white')

### 修正：パターン分析（定格で正規化のみ）入力比<span style="color: red; ">★</span>

In [ ]:
## データID
n_clusters     =  4
outliers_id    =  1
learning_label =  0

#各ラベル定義
Year = '2021'
season_sta = [Year+'-04-01 00:00', Year+'-06-01 00:00', Year+'-10-01 00:00', Year+'-12-01 00:00']
season_fin = [Year+'-05-31 23:00', Year+'-09-30 23:00', Year+'-11-30 23:00', str(int(Year)+1)+'-03-31 23:00']

#使用データ：Year_norm

In [ ]:
## データセット指定
DataSet = pd.read_csv(Pass_In+Input_File+'入力比データ.csv',encoding='cp932',index_col=0,parse_dates=True).fillna(0)
##各種ラベル名
EHP_name = ['EHPK-1','EHPK-2','EHPK-3','GHEX-1','GHEX-2','GHEX-3','GHEX-4','GHEX-5','EHP1-1','EHP1-2','EHP1-3',
 'EHP2-1','EHP2-2','EHP2-3','EHP2-4','EHP2-5','EHP2-6','EHP3-1','EHP3-2','EHP3-3','EHP3-4','EHP3-5','EHP3-6',
 'EHP4-1','EHP4-2','EHP4-3','EHP4-4','EHP4-5','EHP4-6','EHP5-1','EHP5-2','EHP5-3',
 'EHP6-1','EHP6-2','EHP6-3']
## 空調機ラベル
G_LABEL = EHP_name

## データインデックス
_from = season_sta[0]
_to   = season_fin[3]

date_index = pd.date_range(start=_from, end=_to, freq='H')

## 分析データ処理
DataSet['日時'] = pd.to_datetime(DataSet.index)
#DataSet['日付'] = pd.to_datetime(DataSet.index)
DataSet['日付'] = DataSet['日時'].dt.date
DataSet = DataSet[(_from<=DataSet['日時'])&(DataSet['日時']<=_to)]

In [ ]:
##祝日作り
import jpholiday
analys_day = []
for i in range(int(len(date_index)/24)):
    analys_day.append(date_index[0].date() + datetime.timedelta(days=i)) #analys_day = "yyyy, mm, dd" (date_indexの最初の日付から数える)

## DATE = "yyyy, mm, dd"  日付は８桁文字列の形式
def isBizDay(DATE):
        #Date = datetime.date(int(DATE[0:4]), int(DATE[4:6]), int(DATE[6:8]))
    Date = DATE
    if Date.weekday() >= 5 or jpholiday.is_holiday(Date):
        return 0
    else:
        return 1    

## 日単位
holiday_list_day = [] #日にちの0or1データ
for i in range(len(analys_day)):
    s = isBizDay(analys_day[i])
    holiday_list_day.append(s)

## 時間単位
holiday_list_hour = [] #時間の0or1データ
for s in range(len(holiday_list_day)):
    for i in range(24):
        holiday_list_hour.append(holiday_list_day[s])

DataSet['休日'] = holiday_list_hour

In [ ]:
## 各パターン作成
pat_label = ['休日','夏期','冬期','夏期空調日','冬期空調日','非空調日','ピーク発生日']
pat_label = ['休日','夏期空調日','冬期空調日','非空調日','ピーク発生日']
pat_label = ['休日','夏期空調日','冬期空調日','ピーク発生日']
pat_label = ['休日','夏期','冬期','ピーク発生日']
pat_label = ['休日','夏期空調日','冬期空調日','夏期ピーク発生日','冬期ピーク発生日']
#pat_label = ['休日','夏期空調日','冬期空調日','冬期ピーク発生日']

season_s = [Year+'-04-01 00:00', Year+'-06-01 00:00', Year+'-10-01 00:00', str(int(Year)+1)+'-01-01 00:00']
season_f = [Year+'-05-31 23:00', Year+'-08-30 23:00', Year+'-11-30 23:00', str(int(Year)+1)+'-02-28 23:00']

DataSet_pat = []

## 休日
Frame = pd.DataFrame()
for i in range(24):
    DataSet_holi = DataSet[DataSet['休日']==0]
    Frame[i] = DataSet_holi.set_index('日時').loc[time(i)].mean()[G_LABEL].round(2)
DataSet_pat.append(Frame.T)

##夏期平日空調日
Frame = pd.DataFrame()
data = DataSet[DataSet['休日']==1][(season_s[1]<=DataSet['日時'])&(DataSet['日時']<=season_f[1])].fillna(0)

for s in range(len(G_LABEL)):
    index = data[G_LABEL[s]][data[G_LABEL[s]] >=0.05].index
    index = pd.DatetimeIndex(index.date)
    index = list(set(index))
    R = pd.DataFrame()
    for i in range(len(index)):
        a = data[data['日付'] == index[i]][G_LABEL[s]]
        R = pd.concat([R,a])
        A = []
        for i in range(24):
            r = R.loc[time(i)].mean()[0]
            A.append(r)
    Frame[G_LABEL[s]] = A
Frame = Frame.T.round(2).fillna(0)
DataSet_pat.append(Frame.T) 

##冬期平日空調日
Frame = pd.DataFrame()
data = DataSet[DataSet['休日']==1][(season_s[3]<=DataSet['日時'])&(DataSet['日時']<=season_f[3])].fillna(0)

for s in range(len(G_LABEL)):
    index = data[G_LABEL[s]][data[G_LABEL[s]] >=0.05].index
    index = pd.DatetimeIndex(index.date)
    index = list(set(index))
    R = pd.DataFrame()
    for i in range(len(index)):
        a = data[data['日付'] == index[i]][G_LABEL[s]]
        R = pd.concat([R,a])
        A = []
        for i in range(24):
            r = R.loc[time(i)].mean()[0]
            A.append(r)
    Frame[G_LABEL[s]] = A
Frame = Frame.T.round(2).fillna(0)
DataSet_pat.append(Frame.T) 

"""
## 夏期(平日)
Frame = pd.DataFrame()
for i in range(24):
    DataSet_work = DataSet[DataSet['休日']==1][(season_s[1]<=DataSet['日時'])&(DataSet['日時']<=season_f[1])]
    Frame[i] = DataSet_work.set_index('日時').loc[time(i)].mean()[G_LABEL].round(2)
DataSet_pat.append(Frame.T)

## 冬期（平日）
Frame = pd.DataFrame()
for i in range(24):
    DataSet_work = DataSet[DataSet['休日']==1][(season_s[3]<=DataSet['日時'])&(DataSet['日時']<=season_f[3])]
    Frame[i] = DataSet_work.set_index('日時').loc[time(i)].mean()[G_LABEL].round(2)
DataSet_pat.append(Frame.T)


#空調日(8月)
Month_sta = [Year+'-07-01 00:00', Year+'-08-01 00:00', Year+'-9-01 00:00', str(int(Year)+1)+'-01-01 23:00',str(int(Year)+1)+'-02-01 23:00']
Month_fin = [Year+'-07-31 23:00', Year+'-08-31 23:00', Year+'-9-30 23:00', str(int(Year)+1)+'-01-31 23:00',str(int(Year)+1)+'-02-28 23:00']

Frame = pd.DataFrame()
#data = DataSet[(Month_sta[1]<=DataSet['日時'])&(DataSet['日時']<=Month_fin[1])].fillna(0)#正規化済み
data = DataSet[(season_s[1]<=DataSet['日時'])&(DataSet['日時']<=season_f[1])].fillna(0)

for s in range(len(G_LABEL)):
    A = []
    index = data[G_LABEL[s]][data[G_LABEL[s]] >=0.05].index#各設備入力比（正規化電力）平均以上発揮した日を「空調日」として抽出
    a = DataSet.T[index].T[G_LABEL[s]]
    for i in range(24):
        r = a.loc[time(i)].mean()
        A.append(r)
    Frame[G_LABEL[s]] = A
Frame = Frame.T.round(2).fillna(0)
DataSet_pat.append(Frame.T) 

#空調日(2月)
Frame = pd.DataFrame()
#data = DataSet[(Month_sta[3]<=DataSet['日時'])&(DataSet['日時']<=Month_fin[3])].fillna(0)#正規化済み
data = DataSet[(season_s[3]<=DataSet['日時'])&(DataSet['日時']<=season_f[3])].fillna(0)
for s in range(len(G_LABEL)):
    A = []
    index = data[G_LABEL[s]][data[G_LABEL[s]] >=0.05].index#各設備入力比（正規化電力）平均以上発揮した日を「空調日」として抽出
    a = DataSet.T[index].T[G_LABEL[s]]
    for i in range(24):
        r = a.loc[time(i)].mean()
        A.append(r)
    Frame[G_LABEL[s]] = A
Frame = Frame.T.round(2).fillna(0)
DataSet_pat.append(Frame.T) 

#非空調日2
Frame = pd.DataFrame()
data = DataSet.fillna(method='bfill')#正規化済み
#data = EHP_count.resample('D').sum()[G_LABEL]#1が空調日0が非空調日
#B=pd.DataFrame(index=list(range(24)))
for s in range(len(G_LABEL)):
    A = []
    index = data[G_LABEL[s]][data[G_LABEL[s]] <0.05].index#各設備入力比（正規化電力）平均以上発揮した日を「空調日」として抽出
    a = DataSet.T[index].T[G_LABEL[s]]
    for i in range(24):
        r = a.loc[time(i)].mean()#.round(3)
        A.append(r)
    Frame[G_LABEL[s]] = A
Frame = Frame.T.round(2).fillna(0)
DataSet_pat.append(Frame.T)

## ピーク発生日
Frame = pd.DataFrame()
for s in range(len(G_LABEL)):
    data = DataSet.set_index('日時').resample('D').max()[G_LABEL[s]]
    index = data[data == data.max()].index[0]
    Frame[G_LABEL[s]] = DataSet[DataSet['日付'] == index][G_LABEL[s]].reset_index(drop=True)[0:24]
DataSet_pat.append(Frame)
"""

##夏期ピーク発生日
Frame = pd.DataFrame()
for s in range(len(G_LABEL)):
    data = DataSet[(season_s[1]<=DataSet['日時'])&(DataSet['日時']<=season_f[1])].resample('D').max()[G_LABEL[s]]
    index = data[data == data.max()].index[0]
    Frame[G_LABEL[s]] = DataSet[DataSet['日付'] == index][G_LABEL[s]].reset_index(drop=True)[0:24]
DataSet_pat.append(Frame)

##冬期ピーク発生日
Frame = pd.DataFrame()
for s in range(len(G_LABEL)):
    data = DataSet[(season_s[3]<=DataSet['日時'])&(DataSet['日時']<=season_f[3])].resample('D').max()[G_LABEL[s]]
    index = data[data == data.max()].index[0]
    Frame[G_LABEL[s]] = DataSet[DataSet['日付'] == index][G_LABEL[s]].reset_index(drop=True)[0:24]
DataSet_pat.append(Frame)


'''
## 正規化
for i in range(len(G_LABEL)):
    ## 上位0.1%の平均で除算
    a05 = DataSet[G_LABEL[i]].sort_values(ascending=False).head(int(len(DataSet[G_LABEL])*0.001)).mean()
    for s in range(len(DataSet_pat)):
        DataSet_pat[s][G_LABEL[i]] = (DataSet_pat[s][G_LABEL[i]]/a05).round(3)
'''
#ここでの正規化は必要？？

In [ ]:
## 分析結果
## クラスタリング用データセット
DataSet_clust = pd.DataFrame()
for i in range(len(DataSet_pat)):
    DataSet_clust = pd.concat([DataSet_clust, DataSet_pat[i]])
DataSet_clust = DataSet_clust.reset_index(drop=True)
Frame = DataSet_clust.T.fillna(0)

In [ ]:
## クラスタリング★★★
Result_km = pd.read_csv(Pass_In+Output_File+out_clust_data+clust_number1+pattern_1+'結果固定.csv',encoding='cp932')
for s in range(n_clusters):
    name = Result_km[Result_km['クラスタ番号'] == s]
    display(name)

In [ ]:
## グラフ
fig = plt.figure(figsize=(8,10),dpi=200, tight_layout=True)
for i in range(n_clusters):
    ax=fig.add_subplot(321+i)
    graph_fig(fig, ax)
    for s in range(len(DataSet_pat)):
        build_clust = Result_km[Result_km['クラスタ番号']==i]['設備名']
        Data = DataSet_pat[s][build_clust].T.mean()
        ax.plot(Data.index, Data, color=dark_c[s %len(dark_c)], label=pat_label[s], linewidth=3,linestyle="dotted", 
                 marker=markers[0], markersize=6, markeredgewidth=1, markeredgecolor=dark_c[s %len(dark_c)], markerfacecolor=light_c[s %len(light_c)], alpha=0.7)
    plt.text(1, 1.3, 'No.'+str(i), fontsize=16)
    plt.hlines(1, 0, 24,colors='red', linestyle='dashed', linewidth=1)
    graph_xy(ax, 15, '時間', '入力比', 0,24, 0,1.6)
    graph_axis(ax)


## 凡例のみ表示
fig, ax0 = plt.subplots(ncols=1, figsize=(10,0.1),dpi=200)
for s in range(len(DataSet_pat)):
    ax0.plot(Data.index, Data, color=dark_c[s %len(dark_c)], label=pat_label[s], linewidth=3,linestyle="dotted", 
             marker=markers[0], markersize=7.5, markeredgewidth=1, markeredgecolor=dark_c[s %len(dark_c)], markerfacecolor=light_c[s %len(light_c)], alpha=0.7)
graph_none(fig, ax0)
axlegend1 = ax0.legend(bbox_to_anchor=(0.5, 0.5), loc='center', ncol=1, frameon = True, fontsize = 15, numpoints = 1,facecolor='white')

#### 重点的対策空調機抽出のためのバブル図 

In [ ]:
##データ準備
Holi = DataSet_pat[0].mean()
Sum = DataSet_pat[1].mean()
Win = DataSet_pat[2].mean()
Sum_p = DataSet_pat[3].mean()
Win_p = DataSet_pat[4].mean()

mean = (Holi+Sum+Win+Sum_p+Win_p)/5
col = pat_label
FRAME = pd.concat([Holi,Sum,Win,Sum_p,Win_p],axis=1)
FRAME.columns=col
FRAME['入力比平均'] = mean
COL = ['入力比平均','休日', '夏期空調日', '冬期空調日', '夏期ピーク発生日', '冬期ピーク発生日']
FRAME = FRAME[COL]

In [ ]:
##類型化結果反映
##データ読み込み
Result_km = pd.read_csv(Pass_In+Output_File+out_clust_data+clust_number1+pattern_1+'結果固定.csv',encoding='cp932',index_col=0)
Per = FRAME
X=['全期間\n入力比平均','休日入力比', '夏期空調日\n入力比', '冬期空調日\n入力比', '夏期ピーク発生日\n入力比', '冬期ピーク発生日\n入力比']
# 正規化データ(DataをData_Scaleに変えただけ)(全体のピーク値を1.0)
Per_Scale = pd.DataFrame()
Per_Scale = (Per - Per.values.min()) / (Per.values.max() - Per.values.min())
##データ並び替え
#Per_Scale = Per_Scale.sort_values('入力比平均', ascending=True)
#Per_Scale = Per_Scale.sort_values('冬期ピーク発生日', ascending=True)
#Per_Scale = Per_Scale.sort_values('夏期ピーク発生日', ascending=True)
#Per_Scale = Per_Scale.sort_values('夏期空調日', ascending=True)
#Per_Scale = Per_Scale.sort_values('冬期空調日', ascending=True)

#Result_km = Result_km.set_index('建物名',drop=True)
Result_km = Result_km.T[Per_Scale.index].T
R = pd.concat([Result_km[Result_km['クラスタ番号']==1],Result_km[Result_km['クラスタ番号']==3]],axis=0)#.reset_index()
Per_Scale = Per_Scale.T[R.index].T
R = R.reset_index()
Per_Scale = Per_Scale.sort_values('入力比平均', ascending=True)
R = R.set_index('設備名').T[Per_Scale.index].T
#バブルプロット
#4つの変数（x軸、y軸、バブルの大きさ、バブルの色）を描画できる！
col = ['Tomato', 'MediumPurple', 'lightGreen','DodgerBlue']
col2 = ['OrangeRed', 'Indigo', 'Green','darkblue']
lab =  ['No.0', 'No.1', 'No.2','No.3']
lab =  ['No.1','No.3']

plt.rcParams["font.size"]=12
fig = plt.figure(figsize=(10,9),dpi=200)
mark_s = [0.0005]

for energy in range(len(Per_Scale.columns[::-1])):
    for build in range(len(Per_Scale.index)):
        plt.scatter(X[energy], Per_Scale.index[build],s=Per_Scale[Per_Scale.columns[energy]][Per_Scale.index[build]]/mark_s[energy %len(mark_s)], 
                    alpha=0.5, linewidths=2,color=col[R['クラスタ番号'][build]],
                    edgecolors=col2[R['クラスタ番号'][build]])
        
plt.grid(color='gray', linestyle='dotted', linewidth=1)     
#plt.title("エネルギー種別\n原単位[MJ/m2・年]")
plt.xlabel("")
plt.ylabel("空調室外機名") 


##凡例は別で作成
fig, ax0 = plt.subplots(ncols=1, figsize=(10,2),dpi=200)
size = [200,200,200,200]

Col = ['MediumPurple', 'DodgerBlue']
Col2 = ['Indigo','darkblue']
for s in range(2):
    plt.scatter(0,0,s=size[s],facecolor=Col[s],edgecolor=Col2[s],label=lab[s], alpha=0.7, linewidths=2)
plt.legend(ncol=4,title='', bbox_to_anchor=(1.35, 1), loc='upper left',labelspacing=1,borderpad=1)
graph_none(fig, ax0)


#### 修正

In [ ]:
##データ準備(項目減らしたver)
Holi = DataSet_pat[0].mean()
Sum = DataSet_pat[1].mean()
Win = DataSet_pat[2].mean()
#Sum_p = DataSet_pat[3].mean()
#Win_p = DataSet_pat[4].mean()

mean = (Holi+Sum+Win)/3
col = pat_label
FRAME = pd.concat([Holi,Sum,Win],axis=1)
FRAME.columns=['休日', '夏期空調日', '冬期空調日']
FRAME['入力比平均'] = mean
COL = ['入力比平均','休日', '夏期空調日', '冬期空調日']
FRAME = FRAME[COL]

In [ ]:
##フロア類型化結果反映
##データ読み込み
Result_km = pd.read_csv(Pass_In+Output_File+out_clust_data+clust_number1+pattern_1+'result.csv',encoding='cp932',index_col=1)
Per = FRAME
X=['入力比平均','休日', '夏期空調日', '冬期空調日']
# 正規化データ(DataをData_Scaleに変えただけ)(全体のピーク値を1.0)
Per_Scale = pd.DataFrame()
Per_Scale = (Per - Per.values.min()) / (Per.values.max() - Per.values.min())
##データ並び替え
Per_Scale = Per_Scale.sort_values('入力比平均', ascending=True)
#Per_Scale = Per_Scale.sort_values('冬期ピーク発生日', ascending=True)

#Result_km = Result_km.set_index('建物名',drop=True)
Result_km = Result_km.T[Per_Scale.index].T

#バブルプロット
#4つの変数（x軸、y軸、バブルの大きさ、バブルの色）を描画できる！
col = ['Tomato', 'MediumPurple', 'lightGreen','DodgerBlue']
col2 = ['OrangeRed', 'Indigo', 'Green','darkblue']
lab =  ['No.0', 'No.1', 'No.2','No.3']

plt.rcParams["font.size"]=16
fig = plt.figure(figsize=(11,11),dpi=200)
mark_s = [0.0005]

for energy in range(len(Per_Scale.columns[::-1])):
    for build in range(len(Per_Scale.index)):
        plt.scatter(X[energy], Per_Scale.index[build],s=Per_Scale[Per_Scale.columns[energy]][Per_Scale.index[build]]/mark_s[energy %len(mark_s)], 
                    alpha=0.5, linewidths=2,color=col[Result_km['クラスタ番号'][build]],
                    edgecolors=col2[Result_km['クラスタ番号'][build]])
        
plt.grid(color='gray', linestyle='dotted', linewidth=1)     
#plt.title("エネルギー種別\n原単位[MJ/m2・年]")
plt.xlabel("")
plt.ylabel("空調室外機名") 

##凡例は別で作成
fig, ax0 = plt.subplots(ncols=1, figsize=(10,2),dpi=200)
size = [200,200,200,200]
for s in range(4):
    plt.scatter(0,0,s=size[s],facecolor=col[s],edgecolor=col2[s],label=lab[s], alpha=0.7, linewidths=2)
plt.legend(ncol=4,title='', bbox_to_anchor=(1.35, 1), loc='upper left',labelspacing=1,borderpad=1)
graph_none(fig, ax0)

#### 全空調機グラフ化 <span style="color: red; ">★</span>

In [ ]:
## グラフ
fig = plt.figure(figsize=(9,25),dpi=200, tight_layout=True)
R = Result_km.T[EHP_name].T
for i in range(len(G_LABEL)):
    ax=fig.add_subplot(12,3,1+i)
    graph_fig(fig, ax)
    for s in range(len(DataSet_pat)):
        build_clust = R.index[i]
        Data = DataSet_pat[s][build_clust]#.T.mean()
        ax.plot(Data.index, Data, color=dark_c[s %len(dark_c)], label=pat_label[s], linewidth=3,linestyle="dotted", 
                 marker=markers[0], markersize=4, markeredgewidth=1, markeredgecolor=dark_c[s %len(dark_c)], markerfacecolor=light_c[s %len(light_c)], alpha=0.7)
    plt.text(1, 1.5, R.index[i], fontsize=10)
    plt.hlines(1, 0, 24,colors='red', linestyle='dashed', linewidth=1)
    graph_xy(ax, 10, '時間', '入力比', 0,24, 0,1.7)
    graph_axis(ax)


## 凡例のみ表示
fig, ax0 = plt.subplots(ncols=1, figsize=(10,0.1),dpi=200)
for s in range(len(DataSet_pat)):
    ax0.plot(Data.index, Data, color=dark_c[s %len(dark_c)], label=pat_label[s], linewidth=3,linestyle="dotted", 
             marker=markers[0], markersize=7.5, markeredgewidth=1, markeredgecolor=dark_c[s %len(dark_c)], markerfacecolor=light_c[s %len(light_c)], alpha=0.7)
graph_none(fig, ax0)
axlegend1 = ax0.legend(bbox_to_anchor=(0.5, 0.5), loc='center', ncol=7, frameon = True, fontsize = 15, numpoints = 1,facecolor='white')

### 特徴量分析

In [ ]:
##クラスタリング結果読み込み
Result = pd.read_csv(Pass_In+Output_File+out_clust_data+clust_number1+pattern_1+'result.csv',encoding='cp932',index_col=0)
##特徴量データ読み込み
label = pd.read_csv(Pass_In+Input_File+Input_Feature+"年間空調機別特徴量データ(2021).csv",index_col=0)

In [ ]:
##カラム名変更
label.columns = ['型番名', '型番', '方角', '方角(cos)', '方角(sin)', '方角(tan)', 'フロア', '用途名', '用途',
       '冷房定格電力', '50%容量時定格冷房電力', '暖房定格電力', '冷房定格能力', '50%容量時定格冷房能力',
       '暖房定格能力', '空調面積', '年平均設定温度', '年最高設定温度', '年最低設定温度', '年最高-年平均設定温度差',
       '年平均-年最低設定温度差', '年最高-年最低設定温度差', '年平均室内温度', '年最高室内温度', '年最低室内温度',
       '年最高-年平均室内温度差', '年平均-年最低室内温度差', '年最高-年最低室内温度差', '夏期空調時間数',
       '夏期全負荷運転時間数', '冬期空調時間数', '冬期全負荷運転時間数','夏期平均設定温度','冬期平均設定温度']

In [ ]:
#必要な特徴量の抽出
Label_A = label[[
    #'型番名',
    #'型番',
    #'方角',
    #'方角(cos)', 
    #'方角(sin)',
    #'方角(tan)', 
    #'フロア',
    #'用途名',
    '用途',
    '冷房定格電力',
    #'50%容量時定格冷房電力',
    '暖房定格電力',
    #'冷房定格能力',
    #'50%容量時定格冷房能力',
    #'暖房定格能力',
    '空調面積',
    #'年平均設定温度',
    #'年最高設定温度',
    #'年最低設定温度',
    #'年最高-年平均設定温度差',
    #'年平均-年最低設定温度差',
    #'年最高-年最低設定温度差',
    #'年平均室内温度',
    #'年最高室内温度',
    #'年最低室内温度',
    #'年最高-年平均室内温度差',
    #'年平均-年最低室内温度差',
    #'年最高-年最低室内温度差',
    '夏期空調時間数',
    '夏期全負荷運転時間数',
    '冬期空調時間数',
    '冬期全負荷運転時間数',
    '夏期平均設定温度',
    '冬期平均設定温度'
]]

graph_label= [
    #'型番名',
    #'型番',
    #'方角',
    #'方角(cos)', 
    #'方角(sin)',
    #'方角(tan)', 
    #'フロア',
    #'用途名',
    '用途',
    '冷房\n定格\n電力',
    #'50%容量時\n定格\n冷房電力',
    '暖房\n定格\n電力',
   # '冷房\n定格\n能力',
    #'50%容量時\n定格\n冷房能力',
    #'暖房\n定格\n能力',
    '空調\n面積',
    #'年平均\n設定温度',
    #'年最高\n設定温度',
    #'年最低\n設定温度',
    #'年最高-年平均\n設定\n温度差',
    #'年平均-年最低\n設定\n温度差',
    #'年最高-年最低\n設定\n温度差',
    #'年平均\n室内温度',
    #'年最高\n室内温度',
    #'年最低\n室内温度',
    #'年最高-年平均\n室内\n温度差',
    #'年平均-年最低\n室内\n温度差',
    #'年最高-年最低\n室内\n温度差',
    '夏期\n空調\n時間数',
    '夏期\n全負荷\n運転時間数',
    '冬期\n空調\n時間数',
    '冬期\n全負荷\n運転時間数',
    '夏期平均\n設定温度',
    '冬期平均\n設定温度'
]

In [ ]:
## FIスコア算出
FI_DATA = pd.DataFrame()
f_data = Label_A

#Result_km1 = Result_km1[['建物名','クラスタ番号']]

#for i in range(n_clusters):
for i in range(Result['クラスタ番号'].max()+1):#４回試行（3+1)
    ## 特徴量重要度
    #data_class = pd.get_dummies(Result_km,columns=['クラスタ番号修正']).set_index('建物名').iloc[:,i]
    data_class = pd.get_dummies(Result,columns=['クラスタ番号']).sort_values('設備名',ascending=True).set_index('設備名').iloc[:,i]
    clf = RandomForestClassifier(n_estimators=1000, criterion='gini', max_depth=10, min_samples_leaf=1, max_features='auto')
    clf.fit(f_data, data_class)
    ## データ格納
    fti = clf.feature_importances_  
    FI_DATA['No.'+str(i)] = fti
FI_DATA = FI_DATA.set_index(f_data.columns).T

In [ ]:
## グラフ
df = FI_DATA.iloc[::-1]

plt.rcParams["font.size"]=14
n_rows, n_cols = df.shape
positions = np.arange(n_rows)
offsets = np.zeros(n_rows, dtype=df.values.dtype)
colors = plt.get_cmap("tab20b")(np.linspace(0, 1, n_cols))#0～1をn_colsで割る

fig, ax = plt.subplots(ncols=1, figsize=(16,9),dpi=200)
ax.set_yticks(positions)
ax.set_yticklabels(df.index)
graph_axis2(ax)
graph_x(ax, 13, '特徴量重要度','パターン名称', 0,1)

for i in range(len(df.columns)):
    # 棒グラフを描画する。
    bar = ax.barh(positions, df.iloc[:, i], left=offsets, color=colors[i],label=df.columns[i], height=0.7, alpha=0.6)
    offsets += df.iloc[:, i]
    
    # 棒グラフのラベルを描画する。
    for rect in bar:
        cx = rect.get_x() + rect.get_width() / 2
        cy = rect.get_y() + rect.get_height() / 2
        if rect.get_width()<0.07:
            ax.text(cx, cy, '', color="k", ha="center", va="center")
        else:
            ax.text(cx, cy, graph_label[i], color="k", ha="center", va="center")
        
axlegend1 = ax.legend(bbox_to_anchor=(0.5, 1), loc='lower center', ncol=5, frameon = True, fontsize = 16, numpoints = 1)

In [ ]:
## データ正規化
data_norm = pd.DataFrame(index=f_data.index)
for i in range(len(f_data.columns)):
    data_norm[f_data.columns[i]] = zscore(np.array(f_data.iloc[:,i]).astype('float64'))
data_norm = data_norm[FI_DATA.columns]

Data_Norm = []
for i in range(len(FI_DATA.index)):#4回(クラスタNo.0～3)
    f_data2 = pd.DataFrame(index=data_norm.columns)
    data_class = pd.get_dummies(Result,columns=['クラスタ番号']).set_index('設備名').iloc[:,i]
    f_data2['特徴量重要度'] = FI_DATA.T.iloc[:,i]
    f_data2['クラスタ平均値'] = data_norm.T[data_class[data_class==1].index].T.mean()
    f_data2 = f_data2.sort_values('特徴量重要度',ascending=True).tail(5)#.iloc[4:9,:]##重要度の大きい順で並び替え#上位5項目抽出
    Data_Norm.append(f_data2)

In [ ]:
## グラフ
plt.rcParams["font.size"]=15
for s in range(len(Data_Norm)):
    gs_kw = {'width_ratios':[1,3]}
    fig, ax = plt.subplots(ncols=2, gridspec_kw=gs_kw, figsize=(9,2),dpi=200 ,tight_layout=False, sharey=True)


    ax[0].barh(Data_Norm[s].index, Data_Norm[s].iloc[:,0], fc=light_b[6], ec=dark_b[6], linewidth=2, height=0.6, alpha=0.8)
    ax[1].scatter(Data_Norm[s].iloc[:,1], Data_Norm[s].index, fc=light_b[6], ec=dark_b[6], marker='s', s=90, alpha=0.5, linewidths=2,
                  label='Cluster平均')
    ax[1].plot(np.zeros(len(Data_Norm[s])), Data_Norm[s].index, c=light_b[3], marker='o',markersize=9, markeredgecolor=dark_b[3], alpha=0.4,
               markeredgewidth=2, linestyle=':', linewidth=1, label='全空調機平均(＝0)')

    graph_x(ax[0], 15, '特徴量重要度', '特徴量(説明変数)', 0,0.4)
    graph_x(ax[1], 15, '平均より小 ← 特徴量(正規化) → 平均より大','', -2.0,2.0)
    graph_axis(ax[0])
    graph_axis(ax[1])
    plt.text(-1.8, 3.3, 'No.'+str(s), fontsize=15)

    axlegend1 = ax[1].legend(bbox_to_anchor=(0.36, 0.0), loc='lower right', ncol=1, frameon = True, fontsize = 10, numpoints = 1)
    plt.show()

## パターン分析（上位値で正規化のみ）→波形重視　＋特徴量分析

### パターン分析（上位値で正規化のみ）

In [ ]:
## データID
n_clusters     =  4
outliers_id    =  1
learning_label =  0

#各ラベル定義
Year = '2021'
season_sta = [Year+'-04-01 00:00', Year+'-06-01 00:00', Year+'-10-01 00:00', Year+'-12-01 00:00']
season_fin = [Year+'-05-31 23:00', Year+'-09-30 23:00', Year+'-11-30 23:00', str(int(Year)+1)+'-03-31 23:00']

#使用データ：clust_2021

In [ ]:
## データセット指定
DataSet = clust_2021.fillna(0)
## 空調機ラベル
G_LABEL = EHP_name

## データインデックス
_from = season_sta[0]
_to   = season_fin[3]

date_index = pd.date_range(start=_from, end=_to, freq='H')

## 分析データ処理
DataSet['日時'] = pd.to_datetime(DataSet.index)
#DataSet['日付'] = pd.to_datetime(DataSet.index)
DataSet['日付'] = DataSet['日時'].dt.date
DataSet = DataSet[(_from<=DataSet['日時'])&(DataSet['日時']<=_to)]

In [ ]:
##祝日作り
import jpholiday
analys_day = []
for i in range(int(len(date_index)/24)):
    analys_day.append(date_index[0].date() + datetime.timedelta(days=i)) #analys_day = "yyyy, mm, dd" (date_indexの最初の日付から数える)

## DATE = "yyyy, mm, dd"  日付は８桁文字列の形式
def isBizDay(DATE):
        #Date = datetime.date(int(DATE[0:4]), int(DATE[4:6]), int(DATE[6:8]))
    Date = DATE
    if Date.weekday() >= 5 or jpholiday.is_holiday(Date):
        return 0
    else:
        return 1    

## 日単位
holiday_list_day = [] #日にちの0or1データ
for i in range(len(analys_day)):
    s = isBizDay(analys_day[i])
    holiday_list_day.append(s)

## 時間単位
holiday_list_hour = [] #時間の0or1データ
for s in range(len(holiday_list_day)):
    for i in range(24):
        holiday_list_hour.append(holiday_list_day[s])

DataSet['休日'] = holiday_list_hour

In [ ]:
##
"""
## 各パターン作成
pat_label = ['休日','夏期','冬期','夏期空調日','冬期空調日','非空調日','ピーク発生日']

season_s = [Year+'-04-01 00:00', Year+'-06-01 00:00', Year+'-10-01 00:00', Year+'-01-01 00:00']
season_f = [Year+'-05-31 23:00', Year+'-08-30 23:00', Year+'-11-30 23:00', str(int(Year)+1)+'-02-28 23:00']

DataSet_pat = []

## 休日
Frame = pd.DataFrame()
for i in range(24):
    DataSet_holi = DataSet[DataSet['休日']==0]
    Frame[i] = DataSet_holi.set_index('日時').loc[time(i)].mean()[G_LABEL].round(2)
DataSet_pat.append(Frame.T)

## 夏期
Frame = pd.DataFrame()
for i in range(24):
    DataSet_work = DataSet[DataSet['休日']==1][(season_s[1]<=DataSet['日時'])&(DataSet['日時']<=season_f[1])]
    Frame[i] = DataSet_work.set_index('日時').loc[time(i)].mean()[G_LABEL].round(2)
DataSet_pat.append(Frame.T)

## 冬期
Frame = pd.DataFrame()
for i in range(24):
    DataSet_work = DataSet[DataSet['休日']==1][(season_s[3]<=DataSet['日時'])&(DataSet['日時']<=season_f[3])]
    Frame[i] = DataSet_work.set_index('日時').loc[time(i)].mean()[G_LABEL].round(2)
DataSet_pat.append(Frame.T)

#空調日(8月)
Month_sta = [Year+'-07-01 00:00', Year+'-08-01 00:00', Year+'-9-01 00:00', str(int(Year)+1)+'-01-01 23:00',str(int(Year)+1)+'-02-01 23:00']
Month_fin = [Year+'-07-31 23:00', Year+'-08-31 23:00', Year+'-9-30 23:00', str(int(Year)+1)+'-01-31 23:00',str(int(Year)+1)+'-02-28 23:00']

Frame = pd.DataFrame()
#data = DataSet[(Month_sta[1]<=DataSet['日時'])&(DataSet['日時']<=Month_fin[1])].fillna(0)#正規化済み
data = DataSet[(season_s[1]<=DataSet['日時'])&(DataSet['日時']<=season_f[1])].fillna(0)

for s in range(len(G_LABEL)):
    A = []
    index = data[G_LABEL[s]][data[G_LABEL[s]] >=0.05].index#各設備入力比（正規化電力）平均以上発揮した日を「空調日」として抽出
    a = DataSet.T[index].T[G_LABEL[s]]
    for i in range(24):
        r = a.loc[time(i)].mean()
        A.append(r)
    Frame[G_LABEL[s]] = A
Frame = Frame.T.round(2).fillna(0)
DataSet_pat.append(Frame.T) 

#空調日(2月)
Frame = pd.DataFrame()
#data = DataSet[(Month_sta[3]<=DataSet['日時'])&(DataSet['日時']<=Month_fin[3])].fillna(0)#正規化済み
data = DataSet[(season_s[3]<=DataSet['日時'])&(DataSet['日時']<=season_f[3])].fillna(0)
for s in range(len(G_LABEL)):
    A = []
    index = data[G_LABEL[s]][data[G_LABEL[s]] >=0.05].index#各設備入力比（正規化電力）平均以上発揮した日を「空調日」として抽出
    a = DataSet.T[index].T[G_LABEL[s]]
    for i in range(24):
        r = a.loc[time(i)].mean()
        A.append(r)
    Frame[G_LABEL[s]] = A
Frame = Frame.T.round(2).fillna(0)
DataSet_pat.append(Frame.T) 

#非空調日2
Frame = pd.DataFrame()
data = DataSet.fillna(method='bfill')#正規化済み
#data = EHP_count.resample('D').sum()[G_LABEL]#1が空調日0が非空調日
#B=pd.DataFrame(index=list(range(24)))
for s in range(len(G_LABEL)):
    A = []
    index = data[G_LABEL[s]][data[G_LABEL[s]] <0.05].index#各設備入力比（正規化電力）平均以上発揮した日を「空調日」として抽出
    a = DataSet.T[index].T[G_LABEL[s]]
    for i in range(24):
        r = a.loc[time(i)].mean()#.round(3)
        A.append(r)
    Frame[G_LABEL[s]] = A
Frame = Frame.T.round(2).fillna(0)
DataSet_pat.append(Frame.T)

## ピーク発生日
Frame = pd.DataFrame()
for s in range(len(G_LABEL)):
    data = DataSet.set_index('日時').resample('D').max()[G_LABEL[s]]
    index = data[data == data.max()].index[0]
    Frame[G_LABEL[s]] = DataSet[DataSet['日付'] == index][G_LABEL[s]].reset_index(drop=True)[0:24]
DataSet_pat.append(Frame)


## 正規化
for i in range(len(G_LABEL)):
    ## 上位0.1%の平均で除算
    a05 = DataSet[G_LABEL[i]].sort_values(ascending=False).head(int(len(DataSet[G_LABEL])*0.001)).mean()
    for s in range(len(DataSet_pat)):
        DataSet_pat[s][G_LABEL[i]] = (DataSet_pat[s][G_LABEL[i]]/a05).round(3)

#ここでの正規化は必要？？
"""

In [ ]:
## 各パターン作成
pat_label = ['休日','夏期','冬期','夏期空調日','冬期空調日','非空調日','ピーク発生日']
pat_label = ['休日','夏期空調日','冬期空調日','非空調日','ピーク発生日']
pat_label = ['休日','夏期空調日','冬期空調日','ピーク発生日']
pat_label = ['休日','夏期','冬期','ピーク発生日']
pat_label = ['休日','夏期空調日','冬期空調日','夏期ピーク発生日','冬期ピーク発生日']
#pat_label = ['休日','夏期空調日','冬期空調日','冬期ピーク発生日']

season_s = [Year+'-04-01 00:00', Year+'-06-01 00:00', Year+'-10-01 00:00', Year+'-01-01 00:00']
season_f = [Year+'-05-31 23:00', Year+'-08-30 23:00', Year+'-11-30 23:00', str(int(Year)+1)+'-02-28 23:00']

DataSet_pat = []

## 休日
Frame = pd.DataFrame()
for i in range(24):
    DataSet_holi = DataSet[DataSet['休日']==0]
    Frame[i] = DataSet_holi.set_index('日時').loc[time(i)].mean()[G_LABEL].round(2)
DataSet_pat.append(Frame.T)

##夏期平日空調日
Frame = pd.DataFrame()
data = DataSet[DataSet['休日']==1][(season_s[1]<=DataSet['日時'])&(DataSet['日時']<=season_f[1])].fillna(0)

for s in range(len(G_LABEL)):
    index = data[G_LABEL[s]][data[G_LABEL[s]] >=EHP_list['冷房定格電力kW'][s]*0.05].index
    index = pd.DatetimeIndex(index.date)
    index = list(set(index))
    R = pd.DataFrame()
    for i in range(len(index)):
        a = data[data['日付'] == index[i]][G_LABEL[s]]
        R = pd.concat([R,a])
        A = []
        for i in range(24):
            r = R.loc[time(i)].mean()[0]
            A.append(r)
    Frame[G_LABEL[s]] = A
Frame = Frame.T.round(2).fillna(0)
DataSet_pat.append(Frame.T) 

##冬期平日空調日
Frame = pd.DataFrame()
data = DataSet[DataSet['休日']==1][(season_s[3]<=DataSet['日時'])&(DataSet['日時']<=season_f[3])].fillna(0)

for s in range(len(G_LABEL)):
    index = data[G_LABEL[s]][data[G_LABEL[s]] >=EHP_list['暖房定格電力kW'][s]*0.05].index
    index = pd.DatetimeIndex(index.date)
    index = list(set(index))
    R = pd.DataFrame()
    for i in range(len(index)):
        a = data[data['日付'] == index[i]][G_LABEL[s]]
        R = pd.concat([R,a])
        A = []
        for i in range(24):
            r = R.loc[time(i)].mean()[0]
            A.append(r)
    Frame[G_LABEL[s]] = A
Frame = Frame.T.round(2).fillna(0)
DataSet_pat.append(Frame.T) 

"""
## 夏期(平日)
Frame = pd.DataFrame()
for i in range(24):
    DataSet_work = DataSet[DataSet['休日']==1][(season_s[1]<=DataSet['日時'])&(DataSet['日時']<=season_f[1])]
    Frame[i] = DataSet_work.set_index('日時').loc[time(i)].mean()[G_LABEL].round(2)
DataSet_pat.append(Frame.T)

## 冬期（平日）
Frame = pd.DataFrame()
for i in range(24):
    DataSet_work = DataSet[DataSet['休日']==1][(season_s[3]<=DataSet['日時'])&(DataSet['日時']<=season_f[3])]
    Frame[i] = DataSet_work.set_index('日時').loc[time(i)].mean()[G_LABEL].round(2)
DataSet_pat.append(Frame.T)


#空調日(8月)
Month_sta = [Year+'-07-01 00:00', Year+'-08-01 00:00', Year+'-9-01 00:00', str(int(Year)+1)+'-01-01 23:00',str(int(Year)+1)+'-02-01 23:00']
Month_fin = [Year+'-07-31 23:00', Year+'-08-31 23:00', Year+'-9-30 23:00', str(int(Year)+1)+'-01-31 23:00',str(int(Year)+1)+'-02-28 23:00']

Frame = pd.DataFrame()
#data = DataSet[(Month_sta[1]<=DataSet['日時'])&(DataSet['日時']<=Month_fin[1])].fillna(0)#正規化済み
data = DataSet[(season_s[1]<=DataSet['日時'])&(DataSet['日時']<=season_f[1])].fillna(0)

for s in range(len(G_LABEL)):
    A = []
    index = data[G_LABEL[s]][data[G_LABEL[s]] >=0.05].index#各設備入力比（正規化電力）平均以上発揮した日を「空調日」として抽出
    a = DataSet.T[index].T[G_LABEL[s]]
    for i in range(24):
        r = a.loc[time(i)].mean()
        A.append(r)
    Frame[G_LABEL[s]] = A
Frame = Frame.T.round(2).fillna(0)
DataSet_pat.append(Frame.T) 

#空調日(2月)
Frame = pd.DataFrame()
#data = DataSet[(Month_sta[3]<=DataSet['日時'])&(DataSet['日時']<=Month_fin[3])].fillna(0)#正規化済み
data = DataSet[(season_s[3]<=DataSet['日時'])&(DataSet['日時']<=season_f[3])].fillna(0)
for s in range(len(G_LABEL)):
    A = []
    index = data[G_LABEL[s]][data[G_LABEL[s]] >=0.05].index#各設備入力比（正規化電力）平均以上発揮した日を「空調日」として抽出
    a = DataSet.T[index].T[G_LABEL[s]]
    for i in range(24):
        r = a.loc[time(i)].mean()
        A.append(r)
    Frame[G_LABEL[s]] = A
Frame = Frame.T.round(2).fillna(0)
DataSet_pat.append(Frame.T) 

#非空調日2
Frame = pd.DataFrame()
data = DataSet.fillna(method='bfill')#正規化済み
#data = EHP_count.resample('D').sum()[G_LABEL]#1が空調日0が非空調日
#B=pd.DataFrame(index=list(range(24)))
for s in range(len(G_LABEL)):
    A = []
    index = data[G_LABEL[s]][data[G_LABEL[s]] <0.05].index#各設備入力比（正規化電力）平均以上発揮した日を「空調日」として抽出
    a = DataSet.T[index].T[G_LABEL[s]]
    for i in range(24):
        r = a.loc[time(i)].mean()#.round(3)
        A.append(r)
    Frame[G_LABEL[s]] = A
Frame = Frame.T.round(2).fillna(0)
DataSet_pat.append(Frame.T)

## ピーク発生日
Frame = pd.DataFrame()
for s in range(len(G_LABEL)):
    data = DataSet.set_index('日時').resample('D').max()[G_LABEL[s]]
    index = data[data == data.max()].index[0]
    Frame[G_LABEL[s]] = DataSet[DataSet['日付'] == index][G_LABEL[s]].reset_index(drop=True)[0:24]
DataSet_pat.append(Frame)
"""
##夏期ピーク発生日
Frame = pd.DataFrame()
for s in range(len(G_LABEL)):
    data = DataSet[(season_s[1]<=DataSet['日時'])&(DataSet['日時']<=season_f[1])].resample('D').max()[G_LABEL[s]]
    index = data[data == data.max()].index[0]
    Frame[G_LABEL[s]] = DataSet[DataSet['日付'] == index][G_LABEL[s]].reset_index(drop=True)[0:24]
DataSet_pat.append(Frame)

##冬期ピーク発生日
Frame = pd.DataFrame()
for s in range(len(G_LABEL)):
    data = DataSet[(season_s[3]<=DataSet['日時'])&(DataSet['日時']<=season_f[3])].resample('D').max()[G_LABEL[s]]
    index = data[data == data.max()].index[0]
    Frame[G_LABEL[s]] = DataSet[DataSet['日付'] == index][G_LABEL[s]].reset_index(drop=True)[0:24]
DataSet_pat.append(Frame)


## 正規化
for i in range(len(G_LABEL)):
    ## 上位0.1%の平均で除算
    a05 = DataSet[G_LABEL[i]].sort_values(ascending=False).head(int(len(DataSet[G_LABEL])*0.001)).mean()
    for s in range(len(DataSet_pat)):
        DataSet_pat[s][G_LABEL[i]] = (DataSet_pat[s][G_LABEL[i]]/a05).round(3)

In [ ]:
## 分析結果
## クラスタリング用データセット
DataSet_clust = pd.DataFrame()
for i in range(len(DataSet_pat)):
    DataSet_clust = pd.concat([DataSet_clust, DataSet_pat[i]])
DataSet_clust = DataSet_clust.reset_index(drop=True)
Frame = DataSet_clust.T.fillna(0)

In [ ]:
## クラスタリング
if learning_label == 0:
    km = KMeans(n_clusters=n_clusters,init='k-means++', n_init=10, max_iter=300, tol=1e-04, random_state=0)
elif learning_label == 1:
    km =cluster.SpectralClustering(n_clusters=n_clusters, affinity="nearest_neighbors", n_init=10, random_state=0)
elif learning_label == 2:
    km = cluster.MeanShift()
y_km = km.fit_predict(Frame)
    
Result_km = pd.DataFrame()
Result_km['設備名'] = EHP_name
Result_km['クラスタ番号'] = y_km    
"""
## クラスタ番号の加工    
Cluss, Cluster = [], [1,0,2,3,4]
for i in range(len(Result_km)):
    for s in range(n_clusters):
        if Result_km['クラスタ番号'][i] == s :
            Cluss.append(Cluster[s])
Result_km['クラスタ番号'] = Cluss
"""
Result_km.to_csv(Pass_In+Output_File+out_clust_data+clust_number1+pattern_2+'result.csv',encoding='cp932')
## 表に図示
for s in range(n_clusters):
    name = Result_km[Result_km['クラスタ番号'] == s]
    display(name)
#グラフ用
E = Result_km.sort_values('クラスタ番号').set_index('設備名',drop=True)

In [ ]:
## グラフ
fig = plt.figure(figsize=(8,10),dpi=200, tight_layout=True)
for i in range(n_clusters):
    ax=fig.add_subplot(321+i)
    graph_fig(fig, ax)
    for s in range(len(DataSet_pat)):
        build_clust = Result_km[Result_km['クラスタ番号']==i]['設備名']
        Data = DataSet_pat[s][build_clust].T.mean()
        ax.plot(Data.index, Data, color=dark_c[s %len(dark_c)], label=pat_label[s], linewidth=3,linestyle="dotted", 
                 marker=markers[0], markersize=6, markeredgewidth=1, markeredgecolor=dark_c[s %len(dark_c)], markerfacecolor=light_c[s %len(light_c)], alpha=0.7)
    plt.text(1, 0.85, 'No.'+str(i), fontsize=16)
    graph_xy(ax, 15, '時間', '正規化電力消費', 0,24, 0,1.1)
    graph_axis(ax)


## 凡例のみ表示
fig, ax0 = plt.subplots(ncols=1, figsize=(10,0.1),dpi=200)
for s in range(len(DataSet_pat)):
    ax0.plot(Data.index, Data, color=dark_c[s %len(dark_c)], label=pat_label[s], linewidth=3,linestyle="dotted", 
             marker=markers[0], markersize=7.5, markeredgewidth=1, markeredgecolor=dark_c[s %len(dark_c)], markerfacecolor=light_c[s %len(light_c)], alpha=0.7)
graph_none(fig, ax0)
axlegend1 = ax0.legend(bbox_to_anchor=(0.5, 0.5), loc='center', ncol=7, frameon = True, fontsize = 15, numpoints = 1,facecolor='white')

### 特徴量分析

In [ ]:
##クラスタリング結果読み込み
Result = pd.read_csv(Pass_In+Output_File+out_clust_data+clust_number1+pattern_2+'result.csv',encoding='cp932',index_col=0)
##特徴量データ読み込み
label = pd.read_csv(Pass_In+Input_File+Input_Feature+"年間空調機別特徴量データ(2021).csv",index_col=0)

In [ ]:
##カラム名変更
label.columns = ['型番名', '型番', '方角', '方角(cos)', '方角(sin)', '方角(tan)', 'フロア', '用途名', '用途',
       '冷房定格電力', '50%容量時定格冷房電力', '暖房定格電力', '冷房定格能力', '50%容量時定格冷房能力',
       '暖房定格能力', '処理面積', '年平均設定温度', '年最高設定温度', '年最低設定温度', '年最高-年平均設定温度差',
       '年平均-年最低設定温度差', '年最高-年最低設定温度差', '年平均室内温度', '年最高室内温度', '年最低室内温度',
       '年最高-年平均室内温度差', '年平均-年最低室内温度差', '年最高-年最低室内温度差', '夏期空調時間数',
       '夏期全負荷運転時間数', '冬期空調時間数', '冬期全負荷運転時間数','夏期平均設定温度','冬期平均設定温度']

In [ ]:
#必要な特徴量の抽出
Label_A = label[[
    #'型番名',
    #'型番',
    #'方角',
    #'方角(cos)', 
    #'方角(sin)',
    #'方角(tan)', 
    #'フロア',
    #'用途名',
    #'用途',
    '冷房定格電力',
    #'50%容量時定格冷房電力',
    '暖房定格電力',
    '冷房定格能力',
    #'50%容量時定格冷房能力',
    #'暖房定格能力',
    '処理面積',
    #'年平均設定温度',
    #'年最高設定温度',
    #'年最低設定温度',
    #'年最高-年平均設定温度差',
    #'年平均-年最低設定温度差',
    #'年最高-年最低設定温度差',
    #'年平均室内温度',
    #'年最高室内温度',
    #'年最低室内温度',
    #'年最高-年平均室内温度差',
    #'年平均-年最低室内温度差',
    #'年最高-年最低室内温度差',
    '夏期空調時間数',
    '夏期全負荷運転時間数',
    '冬期空調時間数',
    '冬期全負荷運転時間数',
    '夏期平均設定温度',
    '冬期平均設定温度'
]]

graph_label= [
    #'型番名',
    #'型番',
    #'方角',
    #'方角(cos)', 
    #'方角(sin)',
    #'方角(tan)', 
    #'フロア',
    #'用途名',
    #'用途',
    '冷房\n定格\n電力',
    #'50%容量時\n定格\n冷房電力',
    '暖房\n定格\n電力',
    '冷房\n定格\n能力',
    #'50%容量時\n定格\n冷房能力',
    #'暖房\n定格\n能力',
    '処理\n面積',
    #'年平均\n設定温度',
    #'年最高\n設定温度',
    #'年最低\n設定温度',
    #'年最高-年平均\n設定\n温度差',
    #'年平均-年最低\n設定\n温度差',
    #'年最高-年最低\n設定\n温度差',
    #'年平均\n室内温度',
    #'年最高\n室内温度',
    #'年最低\n室内温度',
    #'年最高-年平均\n室内\n温度差',
    #'年平均-年最低\n室内\n温度差',
    #'年最高-年最低\n室内\n温度差',
    '夏期\n空調\n時間数',
    '夏期\n全負荷\n運転時間数',
    '冬期\n空調\n時間数',
    '冬期\n全負荷\n運転時間数',
    '夏期平均\n設定温度',
    '冬期平均\n設定温度'
]

In [ ]:
## FIスコア算出
FI_DATA = pd.DataFrame()
f_data = Label_A

#Result_km1 = Result_km1[['建物名','クラスタ番号']]

#for i in range(n_clusters):
for i in range(Result['クラスタ番号'].max()+1):#４回試行（3+1)
    ## 特徴量重要度
    #data_class = pd.get_dummies(Result_km,columns=['クラスタ番号修正']).set_index('建物名').iloc[:,i]
    data_class = pd.get_dummies(Result,columns=['クラスタ番号']).sort_values('設備名',ascending=True).set_index('設備名').iloc[:,i]
    clf = RandomForestClassifier(n_estimators=1000, criterion='gini', max_depth=10, min_samples_leaf=1, max_features='auto')
    clf.fit(f_data, data_class)
    ## データ格納
    fti = clf.feature_importances_  
    FI_DATA['No.'+str(i)] = fti
FI_DATA = FI_DATA.set_index(f_data.columns).T

In [ ]:
## グラフ
df = FI_DATA.iloc[::-1]

plt.rcParams["font.size"]=14
n_rows, n_cols = df.shape
positions = np.arange(n_rows)
offsets = np.zeros(n_rows, dtype=df.values.dtype)
colors = plt.get_cmap("tab20b")(np.linspace(0, 1, n_cols))#0～1をn_colsで割る

fig, ax = plt.subplots(ncols=1, figsize=(16,9),dpi=200)
ax.set_yticks(positions)
ax.set_yticklabels(df.index)
graph_axis2(ax)
graph_x(ax, 13, '特徴量重要度','パターン名称', 0,1)

for i in range(len(df.columns)):
    # 棒グラフを描画する。
    bar = ax.barh(positions, df.iloc[:, i], left=offsets, color=colors[i],label=df.columns[i], height=0.7, alpha=0.6)
    offsets += df.iloc[:, i]
    
    # 棒グラフのラベルを描画する。
    for rect in bar:
        cx = rect.get_x() + rect.get_width() / 2
        cy = rect.get_y() + rect.get_height() / 2
        if rect.get_width()<0.07:
            ax.text(cx, cy, '', color="k", ha="center", va="center")
        else:
            ax.text(cx, cy, graph_label[i], color="k", ha="center", va="center")
        
axlegend1 = ax.legend(bbox_to_anchor=(0.5, 1), loc='lower center', ncol=5, frameon = True, fontsize = 16, numpoints = 1)

In [ ]:
## データ正規化
data_norm = pd.DataFrame(index=f_data.index)
for i in range(len(f_data.columns)):
    data_norm[f_data.columns[i]] = zscore(np.array(f_data.iloc[:,i]).astype('float64'))
data_norm = data_norm[FI_DATA.columns]

Data_Norm = []
for i in range(len(FI_DATA.index)):#4回(クラスタNo.0～3)
    f_data2 = pd.DataFrame(index=data_norm.columns)
    data_class = pd.get_dummies(Result,columns=['クラスタ番号']).set_index('設備名').iloc[:,i]
    f_data2['特徴量重要度'] = FI_DATA.T.iloc[:,i]
    f_data2['クラスタ平均値'] = data_norm.T[data_class[data_class==1].index].T.mean()
    f_data2 = f_data2.sort_values('特徴量重要度',ascending=True).tail(5)#.iloc[4:9,:]##重要度の大きい順で並び替え#上位5項目抽出
    Data_Norm.append(f_data2)

In [ ]:
## グラフ
plt.rcParams["font.size"]=15
for s in range(len(Data_Norm)):
    gs_kw = {'width_ratios':[1,3]}
    fig, ax = plt.subplots(ncols=2, gridspec_kw=gs_kw, figsize=(9,2),dpi=200 ,tight_layout=False, sharey=True)


    ax[0].barh(Data_Norm[s].index, Data_Norm[s].iloc[:,0], fc=light_b[6], ec=dark_b[6], linewidth=2, height=0.6, alpha=0.8)
    ax[1].scatter(Data_Norm[s].iloc[:,1], Data_Norm[s].index, fc=light_b[6], ec=dark_b[6], marker='s', s=90, alpha=0.5, linewidths=2,
                  label='Cluster平均')
    ax[1].plot(np.zeros(len(Data_Norm[s])), Data_Norm[s].index, c=light_b[3], marker='o',markersize=9, markeredgecolor=dark_b[3], alpha=0.4,
               markeredgewidth=2, linestyle=':', linewidth=1, label='全空調機平均(＝0)')

    graph_x(ax[0], 15, '特徴量重要度', '特徴量(説明変数)', 0,0.4)
    graph_x(ax[1], 15, '平均より小 ← 特徴量(正規化) → 平均より大','', -2.0,2.0)
    graph_axis(ax[0])
    graph_axis(ax[1])
    plt.text(-1.8, 3.3, 'No.'+str(s), fontsize=15)

    axlegend1 = ax[1].legend(bbox_to_anchor=(1, 0.5), loc='lower right', ncol=1, frameon = True, fontsize = 12, numpoints = 1)
    plt.show()